Idee: Start und Endzustand des Springenden Atoms umbenennen und so tun als wäre das der Sprungzustande

Todo:   - springendes Atom Identifizieren - check
        - neue Daten ausschreiben

In [ ]:
import glob
import os.path


In [ ]:

import ase
from ase.io import read as ASEread
import numpy as np

def identify_jumping_atom(atoms_init, atoms_final):

    # apply minimum image convention to get the displacement vector in direct coordinates for every atom
    init_pos  = atoms_init.get_scaled_positions()
    final_pos = atoms_final.get_scaled_positions()
    for i, (pinit, pfin) in enumerate(zip(init_pos, final_pos)):
        for xyz in [0,1,2]:
            if pfin[xyz] - pinit[xyz] > 0.5:
                final_pos[i][xyz] = final_pos[i][xyz] - 1
            elif pfin[xyz] - pinit[xyz] < -0.5:
                final_pos[i][xyz] = final_pos[i][xyz] + 1
    rel_displacement_vector = final_pos - init_pos
    
    # get cell and compute the displacement vector in cartesian form
    (a,b,c) = atoms_init.get_cell()
    displacement_vector = []
    for (x, y, z) in rel_displacement_vector:
        displacement_vector.append(x*a+y*b+z*c)
    displacement_vector = np.array(displacement_vector)
    
    # get displacement magnitude and check for the ones with displacement larger than 2
    # which should be just the Li that jumps. Get its index
    displacement = np.sqrt(np.sum(displacement_vector ** 2, axis = 1))
    count = 0
    for i, displ in enumerate(displacement):
        if displ > 2:
            count += 1
            Li_index = i
    
    # Check that there are not >1 atoms moving 
    if count != 1:
        print(f"WARNING: There are {count} ions that move more than 2 Ang during the NEB! Check NEB path again!")
    else:
        return Li_index

Aus NEB-folder Struktur 00 und 06 einlesen jumping Li-umbenennen in 00 und aus 06 hinzufügen

In [ ]:
#Li-trans
atoms_for_training_NEB_transition_states = []
H_o_M_for_training_NEB_transition_states = []
paths_for_training_NEB_transition_states = []

# the ones generated manually (0250, 0500, 0750)
paths_for_training_NEB_transition_states  = glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0250/NEB_*_finished/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0500/NEB_*_finished/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_*_finished/run_final")

# the random ones
paths_for_training_NEB_transition_states += glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0*random*/02_odh/NEB_*/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0*random*/03_tsh/NEB_*/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0*random*/04_double_tsh/NEB_*/run_final")

# Iterate over all run_final folders
for path in paths_for_training_NEB_transition_states:
    
    # Check the energy along the path. Use initial and final energies from the corresponding relaxed structures + the last steps of the 
    # optimized intermediate images
    start_structure = ASEread(path.replace('run_final','OUTCAR_initial_image'))
    end_structure = ASEread(path.replace('run_final','OUTCAR_final_image'))
    i = identify_jumping_atom(start_structure, end_structure)
    write_structure = start_structure.copy()
    write_structure[i].symbol = 'Ti'
    end_structure[i].symbol = 'Ti'
    write_structure.append(end_structure[i])
    write_structure.write(path.replace('run_final','jochen_trans_image.vasp'), format='vasp')
    
    
    

In [ ]:
#testing of the data
atoms_for_training_NEB_transition_states = []
H_o_M_for_training_NEB_transition_states = []
paths_for_training_NEB_transition_states = []

# the ones generated manually (0250, 0500, 0750)
paths_for_training_NEB_transition_states  = glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0250/NEB_*_finished/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0500/NEB_*_finished/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_*_finished/run_final")

# the random ones
paths_for_training_NEB_transition_states += glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0*random*/02_odh/NEB_*/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0*random*/03_tsh/NEB_*/run_final")
paths_for_training_NEB_transition_states += glob.glob("/nfshome/winkelmann/ARL/NEBs_Marcel/0*random*/04_double_tsh/NEB_*/run_final")

# Iterate over all run_final folders
for path in paths_for_training_NEB_transition_states:
    
    # Check the energy along the path. Use initial and final energies from the corresponding relaxed structures + the last steps of the 
    # optimized intermediate images
    if os.path.exists(path.replace('run_final','OUTCAR_trans_image')):
        os.rename(path.replace('run_final','OUTCAR_trans_image'),path.replace('run_final','jochen_trans_image.vasp'))
